In [ ]:
#This code imports packages that will be used later on, allowing us to use SeQUeNCe's modules and capabilities

#Import the Node and BSMNode classes, which we will use as the basis for our player's circuits and referees circuit, respectively
from sequence.topology.node import Node, BSMNode

#Import the Memory class, which we will use to hold our qubits
from sequence.components.memory import Memory

#Import the EntanglementGenerationA class, which we will use to entangle our player's qubits
from sequence.entanglement_management.generation import EntanglementGenerationA

#Import the Timeline class, which will allow our simulation to run
from sequence.kernel.timeline import Timeline

#Import the QuantumChannel and ClassicalChannel classes, which allow for communication between Alice and Bob and the referee
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel

#Import the EntanglementProtocol class, which manages the processes of creating entanglement
from sequence.entanglement_management.entanglement_protocol import EntanglementProtocol

#Import the Message class, which enables communication on classical channels
from sequence.message import Message

In [ ]:
#This code defines a custom class for our entangled nodes, as well a resource manager for that class
#The resource manager is needed because SeQUeNCe's implementation of entanglement management requires a resource_manager with an update function

#NEED TO ADD COMMENTS TO THIS CELL
class Manager:
    def __init__(self, node, mem_name):
        self.node = node
        self.mem_name = mem_name
        
    def update(self, prot, mem, st):
        if st == 'RAW':
            mem.reset()
    
    def create_protocol(self, middle: str, other: str):
        self.node.protocols = [EntanglementGenerationA(self.node, '%s.eg' % self.node.name, 
                                                       middle, other, self.node.components[self.mem_name])]

class PlayerNode(Node):
    def __init__(self, name: str, tl: Timeline):
        super().__init__(name, tl)
        
        mem_name = '%s.mem' % name
        mem = Memory(mem_name, tl, fidelity = 1, frequency = 0,
                    efficiency = 1, coherence_time = 0, wavelength = 500)
        mem.owner = self
        mem.add_receiver(self)
        self.add_component(mem)
        self.resource_manager = Manager(self, mem_name)
        
    def init(self):
        mem = self.get_components_by_type('Memory')[0]
        mem.reset()
        
    def receive_msg(self, src: str, msg: 'Message'):
        self.protocols[0].received_message(src, msg)
    
    def get(self, photon, **kwargs):
        self.send_qubit(kwargs['dst'], photon)

In [ ]:
#This code creates the infrastructure for our simulation

#Create the timeline for the simulation
tl = Timeline()

#Create nodes for Alice, Bob, and the ref
alice = PlayerNode('alice', tl)
bob = PlayerNode('bob', tl)

#The referee's channel uses SeQUeNCe's built-in BSMNode
ref = BSMNode('ref', tl, ['alice', 'bob'])

#Set the efficiency of the BSM to 1, which means no errors
bsm = ref.get_components_by_type('SingleAtomBSM')[0]
bsm.update_detectors_params('efficiency', 1)

#Create quantum channels between Alice and Bob and the ref
qcA = QuantumChannel('qcA', tl, attenuation = 0, distance = 1000)
qcB = QuantumChannel('qcB', tl, attenuation = 0, distance = 1000)
qcA.set_ends(alice, ref.name)
qcB.set_ends(bob, ref.name)

#Create classical channels between Alice and Bob and the ref
#Classical channels are one way only, so we have to make two channels for each connection
ccAR = ClassicalChannel('ccAR', tl, distance = 1000, delay = 1e8)
ccBR = ClassicalChannel('ccBR', tl, distance = 1000, delay = 1e8)
ccRA = ClassicalChannel('ccRA', tl, distance = 1000, delay = 1e8)
ccRB = ClassicalChannel('ccRB', tl, distance = 1000, delay = 1e8)
ccAB = ClassicalChannel('ccAB', tl, distance = 1000, delay = 1e8)
ccBA = ClassicalChannel('ccBA', tl, distance = 1000, delay = 1e8)
ccAR.set_ends(alice, ref.name)
ccBR.set_ends(bob, ref.name)
ccRA.set_ends(ref, alice.name)
ccRB.set_ends(ref, bob.name)
ccAB.set_ends(alice, bob.name)
ccBA.set_ends(bob, alice.name)

In [ ]:
#This code creates and pairs the protocols for generating entanglement

#NEED TO ADD COMMENTS TO THIS CELL
def pair_protocol(node1: Node, node2: Node):
    p1 = node1.protocols[0]
    p2 = node2.protocols[0]
    n1_mem_name = node1.get_components_by_type('Memory')[0].name
    n2_mem_name = node2.get_components_by_type('Memory')[0].name
    p1.set_others(p2.name, node2.name, [n2_mem_name])
    p2.set_others(p1.name, node1.name, [n1_mem_name])
    
alice.resource_manager.create_protocol('ref', 'bob')
bob.resource_manager.create_protocol('ref', 'alice')
pair_protocol(alice, bob)

mem = alice.get_components_by_type('Memory')[0]

In [ ]:
#This code runs the simulation
tl.init()
alice.protocols[0].start()
bob.protocols[0].start()
tl.run()

print(mem.entangled_memory)

In [ ]:
#FIGURING OUT GATES AND THEN IMPLEMENTING A REF NODE FOR JUST CLASSICAL COMMUNICATION -- HAVE TO REWORK SOME OF THE ABOVE
from enum import Enum
from sequence import message
import random

class MsgType(Enum):
    ZERO = 0
    ONE = 1
    
class Player(Enum):
    ALICE = 0
    BOB = 1
    
class RefereeProtocol(Protocol):
    def __init__(self, node: Node, name: str, rec_name: str, rec_node: str):
        super().__init__(node, name)
        node.protocols.append(self)
        self.rec_name = rec_name
        self.rec_node = rec_node
        
    def init(self):
        pass
    
    def sendBit(self):
        bit = random.randint(0, 1)
        msg = Message(MsgType(bit), self.rec_name)
        self.node.send_message(self.rec_name, msg)
    
    def received_message(self, src: str, msg: Message):
        print("node {} received {} message from {}").format(self.node.name, msg, src)
        
class PlayerProtocol(Protocol):
    def __init__(self, node: Node, name: str, rec_name: str, rec_node: str):
        super().__init__(node, name)
        node.protocols.append(self)
        self.rec_name = rec_name
        self.rec_node = rec_node
    
    def init(self):
        pass
    
    def received_message(self, src: str, msg: Message):
        #APPLY GATES HERE BY CALLING PROTOCOLS
        print("node {} received {} message from {}").format(self.node.name, msg, src)
        
#DEFINE PROTOCOLS FOR ALICE AND BOB'S GATES
class AliceProtocol(Protocol):
    def __init__(self, node: Node, name: str)

In [ ]:
#WANT TO ADD RESOURCE MANAGER TO RUN IT A FEW TIMES
#WANT TO SHOW HOW CHANGING FIDELITY, USING NETWORK MANAGER, ETC CAN BOOST THEIR SUCCESS